In [1]:
# Use autoplot 207's code for this app
import sys
import datetime
sys.path.insert(0, "/opt/iem/htdocs/plotting/auto/scripts200/")
from p207 import load_data, compute_grid_bounds, add_zeros, do_analysis, USEME
from matplotlib.patches import Rectangle
from shapely.geometry import Point
from pyiem.plot import MapPlot, nwssnow, get_cmap
from pyiem.nws.vtec import NWS_COLORS
from pyiem.util import get_dbconn, utc, get_sqlalchemy_conn
from pyiem.reference import EPSG
from pyproj import Transformer
import numpy as np
import pandas as pd
import geopandas as gpd

In [2]:
STORM_NUMBER = 13
TITLE = "21-22 March 2024"
SUBTITLE = "1 PM 22 March 2024"
SETPOINT_LOCS = {}
sts = utc(2024, 3, 22, 0)
ets = utc(2024, 3, 27, 18)
# Get available data
ctx = {'coop': 'yes', 't': 'state', 'sz': 30, 'z': 'yes', 'f': 'linear', 'v': 'snow', 'wfo': 'DMX'}
df = load_data(ctx, sts, ets)
df = df[~df['nwsli'].isin(['DSXI4', 'DMX'])]
# figure out our grid bounds
ctx["bnds2163"] = compute_grid_bounds(ctx, 'IA')
# add zeros and QC
df = add_zeros(df, ctx)

In [3]:
def overlay_ice(mp):
    """Add plotted ice storm."""
    with get_sqlalchemy_conn("postgis") as conn:
        df = pd.read_sql(
        """
        SELECT st_x(geom) as lon, st_y(geom) as lat, magnitude from lsrs WHERE
        typetext in ('ICE STORM', 'FREEZING RAIN') and magnitude > 0
        and valid > %s and valid < %s and state = 'IA'
        """,
        conn,
        params=(sts - datetime.timedelta(days=1), ets)
        )
    print(df[df["state"] == 'IA'])
    mp.plot_values(df.lon.values, df.lat.values, df.magnitude.values, fmt="%.2f", labelbuffer=1, color='purple')

def workflow(ctx, df, isfinal=False, lower=0, upper=2):
    # do gridding
    df2 = df[df[USEME]]
    lons, lats, vals = do_analysis(df2, ctx)
    mp = MapPlot(
        sector="state",
        state=ctx["csector"],
        axisbg="white",
        title="%s - IEM Snowfall Total Analysis" % (TITLE, ),
        subtitle=("Snowfall totals till %s from NWS COOP, LSR, CoCoRaHS Reports; "
                  "IEM 2022-2023 Winter Storm #%s") % (SUBTITLE, STORM_NUMBER),
        twitter=True,
    )
    cmap = nwssnow()
    # cmap = get_cmap("Greens")
    ramp = [0.1, 1, 2, 3, 4, 6, 8, 12, 18, 24, 30, 36]
    # ramp = [0.1, 1, 2, 3, 4]
    mp.contourf(
        lons, lats, vals, np.array(ramp), cmap=cmap, clip_on=True)
    df_useme_plot = df2[(df2['val']>=lower)&(df2['val'] < upper)]
    print(df[df["state"] == 'IA'])
    mp.drawcounties()
    # overlay_ice(mp)
    if isfinal:
        mp.drawcities()
    else:
        mp.plot_values(df_useme_plot['lon'], df_useme_plot['lat'], df_useme_plot['val'].values,
            '%s', labels=df_useme_plot['nwsli'].values,
            textsize=10, labeltextsize=10, labelbuffer=1)
    return mp

In [4]:
def add_setpoints(setpoints):
    """Manual things."""
    for sp, val in setpoints:
        df.at[10000 + sp, "geo"] = Point(SETPOINT_LOCS[sp][0], SETPOINT_LOCS[sp][1])
        df.at[10000 + sp, "val"] = val
        df.at[10000 + sp, USEME] = True
        df.at[10000 + sp, "plotme"] = True

def draw_setpoints(mp):
    """Add some points where manual obs could be inserted."""
    xlim = mp.panels[0].ax.get_xlim()
    ylim = mp.panels[0].ax.set_ylim()
    sz = ctx["sz"] * 1000.
    i = 0
    trans = Transformer.from_proj(mp.panels[0].crs, 2163, always_xy=True)
    for y in np.arange(ylim[0] + sz/2, ylim[1], sz):
        for x in np.arange(xlim[0] + sz/2, xlim[1], sz):
            mp.panels[0].ax.text(x, y, f"{i}", ha="center", va="center")
            # Need to store the x, y in 2163, which is what p207 uses :/
            (xx, yy) = trans.transform(x, y)
            SETPOINT_LOCS[i] = [xx, yy]
            i += 1

def plotsqw(mp):
    with get_sqlalchemy_conn("postgis") as conn:
        gdf = gpd.read_postgis(
            f"SELECT geom from sbw_{sts.year} w WHERE w.phenomena = 'SQ' and w.issue > %s and w.issue < %s",
            conn,
            params=(sts, ets)
        )
    gdf.to_crs(mp.panels[0].crs).plot(
        ax=mp.panels[0].ax,
        aspect=None,
        edgecolor=NWS_COLORS["SQ.W"],
        facecolor="None",
        zorder=1000,
        linewidth=2,
    )
    p0 = Rectangle((0, 0), 1, 1, ec=NWS_COLORS["SQ.W"], fc="None")
    mp.panels[0].ax.legend((p0, ), ("Snow Squall Warning", ), loc=1).set_zorder(1000)

In [5]:
df

,state,wfo,val,lon,lat,geo,used_for_analysis,nwsli,plotme,source,xcell,ycell
0,NE,LBF,19.0,-100.790000,41.130000,POINT (-66202.232 -429925.089),True,0,True,LSR,-10,3
1,NE,LBF,18.7,-100.770000,41.220000,POINT (-64436.004 -419939.288),True,1,True,LSR,-10,4
2,NE,LBF,17.4,-100.700000,41.130000,POINT (-58660.446 -429993.262),True,2,True,LSR,-10,3
3,NE,LBF,17.0,-100.820000,41.160000,POINT (-68684.105 -426566.689),True,3,True,LSR,-10,3
4,NE,LBF,16.8,-100.700000,41.160000,POINT (-58633.098 -426659.386),True,4,True,LSR,-10,3
...,...,...,...,...,...,...,...,...,...,...,...,...
2327,IL,DVN,0.0,-91.033300,40.883300,POINT (752457.812 -416688.707),True,GLDI2,True,COOP,16,4
2328,IL,DVN,0.0,-89.338100,41.253900,POINT (888671.658 -358796.842),True,HNPI2,True,COOP,20,6
2329,IL,DVN,0.0,-91.016700,41.416700,POINT (747608.492 -357502.409),True,ILNI2,True,COOP,16,6
2330,IL,DVN,0.0,-90.666700,40.466700,POINT (788087.274 -459360.579),True,MQBI2,True,COOP,17,2


In [13]:
def main():
    setpoints = [
    ]
    if setpoints:
        add_setpoints(setpoints)
    cull = [
        'ZRGI4',
        'LWDI4',
        1133,
        'IA-JH-20',
    ]
    if cull:
        df.loc[df['nwsli'].isin(cull), USEME] = False
    hardcode = [
        (1038,4),
        (986,4),
        (937,4),
    ]
    for nwsli, val in hardcode:
        df.loc[df['nwsli'] == nwsli, 'val'] = val

    ctx['csector'] = "IA"
    mp = workflow(ctx, df, isfinal=True, lower=2.0, upper=10.0)
    # draw_setpoints(mp)

    #plotsqw(mp)
    res = mp.postprocess(filename='240322.png')
    mp.close()

main()

     state  wfo  val       lon       lat                             geo  \
260     IA  DVN  8.0 -91.65000  42.16000  POINT (686987.954 -280741.019)   
335     IA  DVN  7.2 -91.29000  42.11000  POINT (717045.231 -283193.414)   
339     IA  DVN  7.0 -91.73000  41.99000  POINT (682287.442 -300237.896)   
347     IA  DVN  7.0 -91.89000  42.12000  POINT (667751.671 -287155.561)   
350     IA  DMX  7.0 -92.58000  41.99000  POINT (612412.838 -306947.737)   
...    ...  ...  ...       ...       ...                             ...   
2950    IA  DMX  0.0 -94.74910  40.98860  POINT (440631.111 -431919.122)   
2951    IA  DMX  0.0 -94.85000  42.06670  POINT (424910.438 -312793.024)   
2952    IA  DMX  0.0 -93.31670  41.00000  POINT (560440.491 -421954.321)   
2953    IA  DMX  0.0 -93.36500  40.70670  POINT (558923.859 -454801.538)   
2954    IA  ARX  0.0 -92.71764  43.28372  POINT (588571.351 -164548.533)   

      used_for_analysis    nwsli  plotme source  xcell  ycell  
260                True